# Depuração Giro de Carteira 2025



In [2]:
%cd "C:\Git\BI0730"
import sys
sys.path.append(r'.\.\.\.\.\.')
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from libs.geral.utils import *
from libs.geral.myconstants import *

print('Versões: pandas', pd.__version__, 'numpy', np.__version__)


C:\Git\BI0730
Versões: pandas 2.3.1 numpy 1.26.4


In [ ]:
ANO = 2025
USER = os.getlogin()

# --- Localização robusta do arquivo de principalidade ---
import re

def localizar_arquivo_principalidade(ano:int, base_parquet_dir:str,
                                     nome_base:str='cia_pcp_indicador_principalidade_historico'):
    """Retorna (caminho_escolhido, lista_arquivos_avaliados).
    Regras:
      1. Se existir diretório <base>/<nome_base>/ com vários parquets:
         - Considera arquivos *.parquet que contenham nome_base.
         - Extrai prefixo inicial AAAAMM antes do primeiro '_'. 
         - Primeiro tenta só os do ano solicitado (AAAAMM começa com str(ano)).
         - Se não houver para o ano, considera todos e pega o maior AAAAMM.
      2. Se não houver diretório, tenta arquivo único <base>/<nome_base>.parquet
    """
    dir_candidatos = os.path.join(base_parquet_dir, nome_base)
    avaliados = []
    selecionaveis = []
    if os.path.isdir(dir_candidatos):
        for f in os.listdir(dir_candidatos):
            if f.lower().endswith('.parquet') and nome_base in f:
                avaliados.append(f)
                m = re.match(r'^(\d{6})_', f)
                if m:
                    aaaamm = m.group(1)
                    try:
                        selecionaveis.append((int(aaaamm), f))
                    except ValueError:
                        pass
        # Filtrar por ano solicitado
        ano_prefix = str(ano)
        por_ano = [t for t in selecionaveis if str(t[0]).startswith(ano_prefix)]
        if por_ano:
            por_ano.sort()
            escolhido = por_ano[-1][1]
        elif selecionaveis:
            selecionaveis.sort()
            escolhido = selecionaveis[-1][1]
        else:
            # Nenhum com padrão AAAAMM_ => fallback arquivo único
            escolhido = None
        if escolhido:
            return os.path.join(dir_candidatos, escolhido), avaliados
        else:
            # fallback arquivo único na raiz
            return os.path.join(base_parquet_dir, f'{nome_base}.parquet'), avaliados
    else:
        # Diretório não existe => assumir arquivo único
        return os.path.join(base_parquet_dir, f'{nome_base}.parquet'), avaliados

principalidade_path, lista_avaliados = localizar_arquivo_principalidade(ANO, PATH_BASES_PARQUET)
print('[principalidade] candidatos avaliados:', lista_avaliados[:8], '...' if len(lista_avaliados) > 8 else '')
print('[principalidade] selecionado:', principalidade_path)

RUTAS = {
    'giro_historico': os.path.join(PATH_BASES_PARQUET, 'giro_sicredi_historico.parquet'),
    'inadimplentes': os.path.join(PATH_BASES_PARQUET, 'base_inadimplentes.parquet'),
    'associados_dir': os.path.join(PATH_BASES_PARQUET, 'associados_total_historico'),
    'saida_dir': fr'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira',
    'principalidade': principalidade_path
}

RUTAS['arquivo_saida'] = os.path.join(RUTAS['saida_dir'], 'giro_de_carteira.parquet')
RUTAS['arquivo_atualizacao'] = os.path.join(RUTAS['saida_dir'], 'giro_de_carteira_atualizacao.parquet')

PARAMS = {
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': 20 - 45,
    'canais_validos': ['AGÊNCIA', 'WHATSAPP', 'MOBI'],
    'principalidade': ['sow']
}

for k, v in RUTAS.items():
    print(f'{k}: {v}')

[principalidade] candidatos avaliados: ['202301_cia_pcp_indicador_principalidade_historico.parquet', '202302_cia_pcp_indicador_principalidade_historico.parquet', '202303_cia_pcp_indicador_principalidade_historico.parquet', '202304_cia_pcp_indicador_principalidade_historico.parquet', '202305_cia_pcp_indicador_principalidade_historico.parquet', '202306_cia_pcp_indicador_principalidade_historico.parquet', '202307_cia_pcp_indicador_principalidade_historico.parquet', '202308_cia_pcp_indicador_principalidade_historico.parquet'] ...
[principalidade] selecionado: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\202506_cia_pcp_indicador_principalidade_historico.parquet
giro_historico: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\giro_sicredi_historico.parquet
inadimplentes: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\base_inadimplentes.parquet
associados_dir

In [4]:
df =  ler_parquet(RUTAS['associados_dir'])

In [5]:
# Funciones auxiliares

def ano_mes_atual():
    return datetime.now().strftime('%Y-%m')

def carregar_parquet_seguro(path):
    if not os.path.exists(path):
        print(f'[AVISO] Arquivo não encontrado: {path}')
        return pd.DataFrame()
    try:
        df = pd.read_parquet(path)
        print(f'[OK] Carregado {path} -> {len(df)} linhas')
        return df
    except Exception as e:
        print(f'[ERRO] Falha lendo {path}: {e}')
        return pd.DataFrame()

In [6]:
# 3. Cargar bases de datos

giro_df_raw = carregar_parquet_seguro(RUTAS['giro_historico'])
# Para associados assumimos vários parquets na pasta (ajustar conforme realidade)
associados_df_list = []
if os.path.isdir(RUTAS['associados_dir']):
    for f in os.listdir(RUTAS['associados_dir']):
        if f.endswith('.parquet'):
            associados_df_list.append(carregar_parquet_seguro(os.path.join(RUTAS['associados_dir'], f)))
associados_df_raw = pd.concat(associados_df_list, ignore_index=True) if associados_df_list else pd.DataFrame()

inadimplentes_df_raw = carregar_parquet_seguro(RUTAS['inadimplentes'])

print('Resumo carregamento:')
print('giro_df_raw:', giro_df_raw.shape)
print('associados_df_raw:', associados_df_raw.shape)
print('inadimplentes_df_raw:', inadimplentes_df_raw.shape)

[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\giro_sicredi_historico.parquet -> 1877846 linhas
[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_total_historico\202501_associados_total_historico.parquet -> 180135 linhas
[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_total_historico\202502_associados_total_historico.parquet -> 183144 linhas
[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_total_historico\202503_associados_total_historico.parquet -> 186234 linhas
[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_total_historico\202504_associados_total_historico.parquet -> 188411 linhas
[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_total_historico\202505_associados_tota

In [7]:
df_princip = carregar_parquet_seguro(RUTAS['principalidade'])
if not df_princip.empty:
    print('Colunas principalidade (parcial):', list(df_princip.columns)[:30])
    print('Total colunas:', len(df_princip.columns), 'linhas:', len(df_princip))
else:
    print('Principalidade vazio ou não encontrado.')

[OK] Carregado C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\202506_cia_pcp_indicador_principalidade_historico.parquet -> 127905 linhas
Colunas principalidade (parcial): ['ano_mes', 'cpf_cnpj', 'segmento', 'cod_cooperativa', 'cod_central', 'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao', 'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 'des_origem', 'vlr_capital_social', 'idade', 'pix_trans_30d', 'cad_pix', 'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia', 'sum_investimentos', 'total_valor_cartao_mercado_scr', 'total_valor_cartao_sicredi', 'total_valor_agro_sicredi', 'total_valor_agro_mercado_scr', 'total_valor_outros_sicredi', 'total_valor_agro_mercado_cpr', 'sow_agro', 'total_valor_outros_mercado_scr', 'sow_outros']
Total colunas: 83 linhas: 127905


In [8]:
print(df_princip)

        ano_mes     cpf_cnpj segmento cod_cooperativa cod_central cod_agencia  \
0        202506  00045563993       PF            0730        7000          20   
1        202506  00368247988       PF            0730        7000          30   
2        202506  00388129913       PF            0730        7000          04   
3        202506  00400360900       PF            0730        7000          32   
4        202506  00600572978       PF            0730        7000          23   
...         ...          ...      ...             ...         ...         ...   
127900   202506  80073307980       PF            0730        7000          10   
127901   202506  80384595987       PF            0730        7000          19   
127902   202506  80667090991       PF            0730        7000          12   
127903   202506  81002840910       PF            0730        7000          10   
127904   202506  84807431900       PF            0730        7000          17   

       cod_carteira status_

In [20]:
# 4a. Processar principalidade (normalização e seleção de colunas úteis)
princip_proc = pd.DataFrame()
if not df_princip.empty:
    tmp = df_princip.copy()
    # Normalizar coluna cpf/cnpj
    cand_id = [c for c in tmp.columns if c.lower() in ['cpf_cnpj','cpf','cnpj','num_cpf_cnpj','nr_cpf_cnpj']]
    if cand_id:
        id_col = cand_id[0]
        tmp['cpf_cnpj'] = (tmp[id_col].astype(str)
                             .str.replace(r'\D','', regex=True)
                             .str.lstrip('0'))
    else:
        print('[principalidade] Nenhuma coluna identificada como cpf_cnpj')
        tmp['cpf_cnpj'] = None

    # Identificar coluna de score
    cand_score = [c for c in tmp.columns if c.lower() in [
        'score_principalidade','score','pontuacao','pontuacao_principalidade','sc_principalidade','indice_principalidade'
    ]]
    score_col = cand_score[0] if cand_score else None

    # Coluna de data / competência
    cand_data = [c for c in tmp.columns if c.lower() in [
        'data_referencia','dt_referencia','dt_ref','competencia','ano_mes','data_competencia'
    ]]
    data_col = cand_data[0] if cand_data else None

    if data_col:
        if data_col == 'ano_mes':
            # tentar converter AAAAMM -> primeiro dia
            tmp['_dt_ref'] = pd.to_datetime(tmp[data_col].astype(str), format='%Y%m', errors='coerce')
        else:
            tmp['_dt_ref'] = pd.to_datetime(tmp[data_col], errors='coerce')
    else:
        tmp['_dt_ref'] = pd.NaT

    # Ordenar por data e manter último registro por cpf_cnpj
    if 'cpf_cnpj' in tmp.columns:
        tmp = (tmp.sort_values('_dt_ref')
                 .drop_duplicates('cpf_cnpj', keep='last'))

    cols_keep = ['cpf_cnpj','_dt_ref']
    if score_col: cols_keep.append(score_col)
    principals_cols_exist = [c for c in cols_keep if c in tmp.columns]
    princip_proc = tmp[principals_cols_exist].rename(columns={
        score_col: 'score_principalidade_real',
        '_dt_ref': 'data_ref_principalidade'
    }) if not tmp.empty else pd.DataFrame()

    print('[principalidade] Colunas detectadas:')
    print(' - id:', cand_id[:1])
    print(' - score:', cand_score[:1])
    print(' - score:', score_col)
    print(' - data:', data_col)
    print('[principalidade] Registros após deduplicação:', len(princip_proc))
else:
    print('[principalidade] DataFrame vazio - nada a processar')

princip_proc.head()

[principalidade] Colunas detectadas:
 - id: ['cpf_cnpj']
 - score: []
 - score: None
 - data: ano_mes
[principalidade] Registros após deduplicação: 127905


,cpf_cnpj,data_ref_principalidade
0,45563993,2025-06-01
85276,6785329159,2025-06-01
85275,6772354966,2025-06-01
85274,6632001998,2025-06-01
85273,6263820926,2025-06-01


In [19]:
cand_score

[]

In [10]:
# 4. Processar giro histórico (filtros)

if not giro_df_raw.empty:
    required = {'ano_mes','cpf_cnpj','data_ultimo_contato','origem','canal'}
    falt = required.difference(giro_df_raw.columns)
    if falt:
        print('Colunas faltantes giro:', falt)
    giro = giro_df_raw.copy()
    giro['ano_mes'] = pd.to_numeric(giro['ano_mes'], errors='coerce')
    giro['data_ultimo_contato'] = pd.to_datetime(giro['data_ultimo_contato'], errors='coerce')
    giro = giro.dropna(subset=['data_ultimo_contato'])
    if 'origemcanal' not in giro.columns:
        giro['origemcanal'] = giro['origem'].astype(str) + giro['canal'].astype(str)
    ano_mes_param = int(ano_mes_atual().replace('-',''))
    giro_f = (giro
              .query('ano_mes == @ano_mes_param')
              .loc[lambda d: d['canal'].isin(PARAMS['canais_validos'])]
              .loc[lambda d: ~d['origemcanal'].isin(PARAMS['excluir_origem_canal'])])
    giro_agg = (giro_f.groupby('cpf_cnpj', as_index=False)
                .agg(data_ultimo_contato=('data_ultimo_contato','max'))
                .assign(ano_mes=ano_mes_param))
else:
    giro_agg = pd.DataFrame()

print('giro_agg shape:', giro_agg.shape)

giro_agg shape: (218775, 3)


In [18]:
# 4b. Filtrar associados estilo BBM

if not associados_df_raw.empty:
    assoc = associados_df_raw.copy()
    if 'des_faixa_risco' in assoc.columns:
        assoc = assoc[assoc['des_faixa_risco'].isin(PARAMS['filtros_risco_bbm'])]
    # saldo_devedor
    if 'saldo_devedor' in assoc.columns:
        assoc = assoc[(assoc['saldo_devedor'].isna()) | (assoc['saldo_devedor'] == 0)]
    # dt_ult_mov
    if 'dt_ult_mov' in assoc.columns:
        data_limite = datetime.now() - timedelta(days=PARAMS['dias_sem_movimentacao'])
        assoc['dt_ult_mov'] = pd.to_datetime(assoc['dt_ult_mov'], errors='coerce')
        assoc = assoc[(assoc['dt_ult_mov'].isna()) | (assoc['dt_ult_mov'] <= data_limite)]
    # correntista ativo
    if {'flg_correntista','flg_ativo'}.issubset(assoc.columns):
        assoc = assoc.query("flg_correntista == 'S' and flg_ativo == 'S'")
else:
    assoc = pd.DataFrame()

print('assoc filtrado shape:', assoc.shape)

assoc filtrado shape: (754937, 20)


In [12]:
assoc

,central,coop,cod_ua,cod_carteira,cpf_cnpj,tipo_pessoa,flg_correntista,flg_ativo,dt_ult_movimento,dt_ini_asso,conta_principal,des_segmento,des_subsegmento,des_publico_estrategico,des_faixa_risco,des_marca,data_competencia,filtro_ano_mes_inicial,filtro_ano_mes_final,dat_criacao_registro
0,7000,0730,08,010200,85816655949,PF,S,S,2025-01-20,2023-08-10,789494,PF,PF II,DEMAIS,BAIXO 2,FISITAL,2025-01-31,202501,202501,2025-02-05 01:43:00.980
1,7000,0730,13,112,54328462920,PF,S,S,2025-01-30,2020-01-09,165826,PF,PF II,60+ ANOS,MÉDIO 1,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980
3,7000,0730,11,331,80244528000145,PJ,S,S,2025-01-31,2017-11-23,328535,PJ,E2,PJ - Pequena,BAIXÍSSIMO,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980
4,7000,0730,50,131,36982604845,PF,S,S,2025-01-31,2024-10-17,413332,PF,PF III,DEMAIS,BAIXO 2,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980
6,7000,0730,05,111,04274143937,PF,S,S,2025-01-22,2003-02-06,263699,PF,PF II,DEMAIS,MÉDIO 1,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315678,7000,0730,19,121,04483147978,PF,S,S,2025-05-26,2021-02-26,844728,PF,PF II,DEMAIS,BAIXO 1,SICREDI,2025-06-30,202506,202506,2025-07-03 02:02:11.800
1315681,7000,0730,40,010101,01582285969,PF,S,S,2025-06-30,2024-02-02,655091,PF,PF I,DEMAIS,BAIXO 2,FISITAL,2025-06-30,202506,202506,2025-07-03 02:02:11.800
1315683,7000,0730,41,010101,09890090929,PF,S,S,2025-06-10,2024-02-21,751525,PF,PF I,MENOR,MÉDIO 2,FISITAL,2025-06-30,202506,202506,2025-07-03 02:02:11.800
1315684,7000,0730,17,221,09823781907,PF,S,S,2025-06-30,2021-09-09,478375,AG,Familiar,DEMAIS,BAIXÍSSIMO,SICREDI,2025-06-30,202506,202506,2025-07-03 02:02:11.800


In [13]:
# 4c. Enriquecer (simulado)

enriq = assoc.copy()
if not enriq.empty:
    # score principalidade
    np.random.seed(42)
    enriq['score_principalidade'] = np.random.randint(1,11, len(enriq))
    # pix cadastrado
    enriq['pix_cadastrado'] = np.random.choice(['Sim','Não'], len(enriq))
    # fluxo_caixa
    def fluxo(row):
        carteira = str(row.get('carteira',''))
        if carteira.startswith('1'):
            return np.random.choice(['Sim','Não'])
        elif carteira.startswith(('2','3')):
            return np.random.choice(['DOMICÍLIO','COBRANÇA','Nenhum'])
        return 'N/A'
    enriq['fluxo_caixa'] = enriq.apply(fluxo, axis=1)
    # mc_6m
    enriq['mc_6m'] = np.random.uniform(0,50000,len(enriq)).round(2)
print('enriquecido shape:', enriq.shape)

enriquecido shape: (754937, 24)


In [14]:
enriq

,central,coop,cod_ua,cod_carteira,cpf_cnpj,tipo_pessoa,flg_correntista,flg_ativo,dt_ult_movimento,dt_ini_asso,conta_principal,des_segmento,des_subsegmento,des_publico_estrategico,des_faixa_risco,des_marca,data_competencia,filtro_ano_mes_inicial,filtro_ano_mes_final,dat_criacao_registro,score_principalidade,pix_cadastrado,fluxo_caixa,mc_6m
0,7000,0730,08,010200,85816655949,PF,S,S,2025-01-20,2023-08-10,789494,PF,PF II,DEMAIS,BAIXO 2,FISITAL,2025-01-31,202501,202501,2025-02-05 01:43:00.980,7,Não,N/A,"45,544.45"
1,7000,0730,13,112,54328462920,PF,S,S,2025-01-30,2020-01-09,165826,PF,PF II,60+ ANOS,MÉDIO 1,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980,4,Sim,N/A,"26,042.99"
3,7000,0730,11,331,80244528000145,PJ,S,S,2025-01-31,2017-11-23,328535,PJ,E2,PJ - Pequena,BAIXÍSSIMO,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980,8,Sim,N/A,"20,996.64"
4,7000,0730,50,131,36982604845,PF,S,S,2025-01-31,2024-10-17,413332,PF,PF III,DEMAIS,BAIXO 2,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980,5,Sim,N/A,"18,026.90"
6,7000,0730,05,111,04274143937,PF,S,S,2025-01-22,2003-02-06,263699,PF,PF II,DEMAIS,MÉDIO 1,SICREDI,2025-01-31,202501,202501,2025-02-05 01:43:00.980,7,Não,N/A,"6,905.66"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315678,7000,0730,19,121,04483147978,PF,S,S,2025-05-26,2021-02-26,844728,PF,PF II,DEMAIS,BAIXO 1,SICREDI,2025-06-30,202506,202506,2025-07-03 02:02:11.800,10,Sim,N/A,"11,884.00"
1315681,7000,0730,40,010101,01582285969,PF,S,S,2025-06-30,2024-02-02,655091,PF,PF I,DEMAIS,BAIXO 2,FISITAL,2025-06-30,202506,202506,2025-07-03 02:02:11.800,10,Sim,N/A,"41,887.09"
1315683,7000,0730,41,010101,09890090929,PF,S,S,2025-06-10,2024-02-21,751525,PF,PF I,MENOR,MÉDIO 2,FISITAL,2025-06-30,202506,202506,2025-07-03 02:02:11.800,1,Não,N/A,"47,867.59"
1315684,7000,0730,17,221,09823781907,PF,S,S,2025-06-30,2021-09-09,478375,AG,Familiar,DEMAIS,BAIXÍSSIMO,SICREDI,2025-06-30,202506,202506,2025-07-03 02:02:11.800,2,Sim,N/A,"9,427.49"


In [15]:
df_base = ler_parquet("base_relacionamento_atual").rename(columns={'num_cpf_cnpj':'cpf_cnpj'})[[]]

In [16]:
df_base

""
0
1
2
3
4
...
180510
180511
180512
180513


In [17]:
pirai = (
    df
    .query("cod_ua == '24'", engine='python')
    .loc[lambda d: d['dt_ini_asso'].notna()]
    .loc[lambda d: d['dt_ini_asso'] >= pd.Timestamp('2024-07-01')]
    .merge(df_base,on="")
)

pirai

KeyError: ''

In [ ]:
# 4d. Mesclar giro + associados + inadimplentes (com normalização cpf_cnpj e debug)

# Normalizar chaves antes do merge para evitar perda de correspondências
if not giro_agg.empty:
    giro_agg['cpf_cnpj'] = (giro_agg['cpf_cnpj'].astype(str)
                             .str.replace(r'\D','', regex=True)
                             .str.lstrip('0'))
    # Garantir datetime
    giro_agg['data_ultimo_contato'] = pd.to_datetime(giro_agg['data_ultimo_contato'], errors='coerce')

if not enriq.empty and 'cpf_cnpj' in enriq.columns:
    enriq['cpf_cnpj'] = (enriq['cpf_cnpj'].astype(str)
                          .str.replace(r'\D','', regex=True)
                          .str.lstrip('0'))

# Debug pré-merge
if not giro_agg.empty:
    print('giro_agg dtypes:')
    print(giro_agg.dtypes[['cpf_cnpj','data_ultimo_contato','ano_mes']])
    print('Registros com data_ultimo_contato não nula em giro_agg:', giro_agg['data_ultimo_contato'].notna().sum())

if not giro_agg.empty and not enriq.empty:
    res = enriq.merge(giro_agg[['cpf_cnpj','data_ultimo_contato','ano_mes']], on='cpf_cnpj', how='left')
else:
    res = pd.DataFrame()

# Pós-merge: reforçar tipos
if not res.empty:
    # Forçar datetime novamente (caso merge degrade para object)
    res['data_ultimo_contato'] = pd.to_datetime(res['data_ultimo_contato'], errors='coerce')
    if 'ano_mes' in res.columns:
        res['ano_mes'] = pd.to_numeric(res['ano_mes'], errors='coerce').astype('Int64')

if not res.empty:
    if 'ano_mes' not in res.columns:
        res['ano_mes'] = int(ano_mes_atual().replace('-',''))
    res['ano_mes'] = res['ano_mes'].fillna(int(ano_mes_atual().replace('-','')))
    res['data_ultimo_giro'] = pd.to_datetime(res['data_ultimo_contato'])
    hoje = pd.Timestamp.now()
    res['dias_sem_giro'] = (hoje - res['data_ultimo_giro']).dt.days.where(res['data_ultimo_giro'].notna())
    if 'data_competencia' not in res.columns:
        res['data_competencia'] = res['data_ultimo_giro'].dt.date
    def cat(d):
        if pd.isna(d): return 'Nunca contatado'
        if d <= 90: return 'Em dia'
        if d <= 180: return 'Atenção'
        if d <= 365: return 'Atrasado'
        return 'Giro Vencido'
    res['categoria_giro'] = res['dias_sem_giro'].apply(cat)
    def prioridade(row):
        p=0
        d=row['dias_sem_giro']
        if pd.isna(d): p+=25
        else: p+=min(d/30,24)
        if 'score_principalidade' in row: p+=row['score_principalidade']
        if 'mc_6m' in row and not pd.isna(row['mc_6m']): p+=row['mc_6m']/10000
        return round(p,2)
    res['prioridade_contato'] = res.apply(prioridade, axis=1)
    res = res.sort_values('prioridade_contato', ascending=False)
    res['giro_vencido'] = np.where((res['dias_sem_giro'].notna()) & (res['dias_sem_giro']>=366),'Sim','Não')

    # inadimplentes
    if not inadimplentes_df_raw.empty and 'cpf_cnpj' in inadimplentes_df_raw.columns:
        inad = inadimplentes_df_raw.copy()
        inad['cpf_cnpj'] = (inad['cpf_cnpj'].astype(str)
                              .str.replace(r'\D','', regex=True)
                              .str.lstrip('0'))
        if 'flg_inadimplente' not in inad.columns:
            inad['flg_inadimplente'] = 'S'
        res = res.merge(inad[['cpf_cnpj','flg_inadimplente']], on='cpf_cnpj', how='left')
        res['flg_inadimplente'] = res['flg_inadimplente'].fillna('N')
    else:
        res['flg_inadimplente'] = 'N'

    # Debug pós-processamento
    print('\nDEBUG pós-merge:')
    print('Total linhas res:', len(res))
    print('data_ultimo_contato dtype:', res['data_ultimo_contato'].dtype)
    print('Não nulos data_ultimo_contato:', res['data_ultimo_contato'].notna().sum())
    print('Exemplos com data_ultimo_contato preenchida:')
    display(res.loc[res['data_ultimo_contato'].notna(), ['cpf_cnpj','data_ultimo_contato','data_ultimo_giro','dias_sem_giro']].head(10))
else:
    print('Resultado vazio após merge')

print('Resultado shape:', res.shape)

In [ ]:
res

In [ ]:
# 5. Guardar resultados e gerar parquet de atualização

if not res.empty:
    os.makedirs(RUTAS['saida_dir'], exist_ok=True)
    res.to_parquet(RUTAS['arquivo_saida'], index=False)
    print('Salvo principal:', RUTAS['arquivo_saida'])

    datas = pd.to_datetime(res['data_competencia'], errors='coerce')
    data_max = datas.max()
    def prox_util(data_ref):
        d = data_ref + timedelta(days=1)
        while d.weekday() >=5:
            d += timedelta(days=1)
        return d
    hoje = datetime.now()
    df_at = pd.DataFrame([{
        'data_atualizacao': data_max.normalize() if pd.notna(data_max) else pd.NaT,
        'proxima_atualizacao': prox_util(hoje).date(),
        'coluna_referencia': 'data_competencia',
        'gerado_em': hoje
    }])
    df_at.to_parquet(RUTAS['arquivo_atualizacao'], index=False)
    print('Salvo atualização:', RUTAS['arquivo_atualizacao'])
else:
    print('Resultado vazio - nada salvo')

In [ ]:
df_at

In [ ]:
# 6. Debug y verificación

if not res.empty:
    print('\nColunas resultado:', len(res.columns))
    print(sorted(res.columns)[:40])
    print('\nTop 5 linhas:')
    display(res.head())
    print('\nDistribuição categoria_giro:')
    print(res['categoria_giro'].value_counts(dropna=False))
    print('\nPrioridade - resumo:')
    print(res['prioridade_contato'].describe())
else:
    print('Resultado vazio para debug')